# Einführung in RDF Schema

Tripelmengen sind unübersichtlich. Es ist generell schwierig einen Einblick zu erhalten worüber eine Tripelmenge Aussagen trifft. Um Tripelmengen und die darin enthaltenen Ressourcen zu organisieren wurde RDF Schema (RDFS) entwickelt. Zentral ist die Gruppierung von Ressourcen als RDFS Klassen. Hat man Klassen definiert, ist es möglich Ressourcen zu typisieren also festlegen, dass eine Ressource eine Instanz einer Klasse ist. Zudem ermöglich RDFS die Erstellung von Klassenhierarchien wie auch Prädikathierarchien. In dieser Übung werden wir diese Konzepte anhand praktischer Beispiele genauer ansehen. Wie üblich, gehen Sie die Codeblöcke nacheinander durch und beantworten Sie die Fragen.

In [2]:
import pandas as pd
from io import BytesIO
from io import StringIO
from rdflib import Graph, URIRef
from rdflib.namespace import RDF
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from rdflib.plugins.sparql import prepareQuery
from IPython.display import display

prefixes = """
@prefix : <http://example.org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
"""

g = Graph()

def evaluate(rs):
    serializer = CSVResultSerializer(rs)
    output = BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(StringIO(output.getvalue().decode())))
    
def query(rdf, sparql):
    r = g.parse(data='{}{}'.format(prefixes, rdf), format='turtle')
    rs = g.query(sparql)
    evaluate(rs)
    
def reason(rdf, sparql):
    superclass = URIRef('http://example.org#Work')
    r = g.parse(data='{}{}'.format(prefixes, rdf), format='turtle')
    rs1 = g.query(prepareQuery('SELECT ?subclass WHERE { ?subclass rdfs:subClassOf ?superclass }'), 
                  initBindings={'superclass': superclass})
    for qs1 in rs1:
        rs2 = g.query(prepareQuery('SELECT ?instance WHERE { ?instance rdf:type ?class }'), 
                      initBindings={'class': qs1['subclass']})
        for qs2 in rs2:
            g.add((qs2['instance'], RDF.type, superclass))
    evaluate(g.query(sparql))

In [5]:
rdf = """
:r1 rdf:type :Album .
:r2 rdf:type :Album .
:r3 rdf:type :Single .
"""

# In wieviele Gruppen kann man die Ressourcen :r1, :r2, :r3 intuitiv organisieren? Antwort: Zwei
# Welche Klassen lassen sich hier rauslesen? Antwort: :Album und :Single

Sie haben nun Klassen identifiziert, diese sind auch mittles URI benannt. 

Erstellen Sie nun als nächstes ein Dokument welches aussagt, dass die identifizierten Klassen in der Tat RDFS Klassen sind.

Führen Sie dann den Codeblock aus damit die SPARQL Abfrage ausgeführt wird.

In [6]:
rdf = """
:Album rdf:type rdfs:Class .
:Single rdf:type rdfs:Class .
"""

query(rdf, "SELECT ?class WHERE { ?class rdf:type rdfs:Class }")

# Entspricht die Antwort der SPARQL Abfrage Ihren Erwartungen? Antwort: Ja

,class
0,http://example.org#Album
1,http://example.org#Single


Verwenden Sie nun Ihre definierten Klassen und typisieren Sie die folgenden Ressourcen `:r1`, `:r2`, und `:r3`.

Die drei Ressourcen sollen jeweils Instanzen einer der Klassen sein.

In [8]:
rdf = """
:Album rdf:type rdfs:Class .
:Single rdf:type rdfs:Class .
:r1 rdf:type :Album .
:r2 rdf:type :Album .
:r3 rdf:type :Single .
:r1 :title "The Dark Side of the Moon" .
:r2 :title "The Wall" .
:r3 :title "What God Wants, Part 1" .
"""

query(rdf, "SELECT ?resource ?class ?title WHERE { ?resource rdf:type ?class . ?resource :title ?title }")

# Entspricht die Antwort der SPARQL Abfrage Ihren Erwartungen? Antwort: Ja

,resource,class,title
0,http://example.org#r2,http://example.org#Album,The Wall
1,http://example.org#r1,http://example.org#Album,The Dark Side of the Moon
2,http://example.org#r3,http://example.org#Single,"What God Wants, Part 1"


In der Vorlesung haben wir gesehen, dass man mit RDFS Unterklassen bilden kann.

Führen Sie nun eine Klasse `:Work` ein als Oberklasse der Klassen die Sie definiert haben.

Die Aussagen sollten festhalten, dass Instanzen Ihrer Klassen *Werke* sind.

Schreiben Sie dann eine SPARQL Abfrage mit der Sie die Unterklassen von `:Work` erhalten.

In [10]:
rdf = """
:Work rdf:type rdfs:Class .
:Album rdfs:subClassOf :Work .
:Single rdfs:subClassOf :Work .
"""

query(rdf, "SELECT ?subClass WHERE { ?subClass rdfs:subClassOf :Work }")

# Entspricht die Antwort der SPARQL Abfrage Ihren Erwartungen? Antwort: Ja

,subClass
0,http://example.org#Single
1,http://example.org#Album


Nun können in RDFS nicht nur Klassen definiert werden sondern auch Prädikate. 

Spezifizieren Sie als nächstes, dass `:title` und `:released` Prädikate sind.

Schreiben Sie dann eine SPARQL Abfrage mit der Sie alle Prädikate erhalten. Stellen Sie dabei sicher, dass die Ausgabe alphabetisch sortiert ist.

In [12]:
rdf = """
:title rdf:type rdf:Property .
:released rdf:type rdf:Property .
"""

query(rdf, "SELECT ?property WHERE { ?property rdf:type rdf:Property } ORDER BY (?property)")

# Entspricht die Antwort der SPARQL Abfrage Ihren Erwartungen? Antwort: Ja

,property
0,http://example.org#released
1,http://example.org#title


RDFS hat zur Folge, dass es zusätzlich zu den *expliziten* Tripel, wie z.B. 

```
:r1 :title "The Dark Side of the Moon" .
```

auch *implizite* Tripel gibt. Solche Tripel sind korrekt (logisch impliziert) stehen aber nicht explizit in der Tripelmenge.

Ein Beispiel. Die Folgenden Tripel sagen aus, dass Max Muster ein Student ist und Studenten Personen sind.

```
:Student rdf:type rdfs:Class .
:Person rdf:type rdfs:Class .
:Student rdfs:subClassOf rdfs:Person .
:r1 rdf:type :Student .
:r1 rdfs:label "Max Muster" .
```

Dies sind explizite Tripel.

Es folgt allerdings, dass Max Muster eine Person ist, als Tripel

```
:r1 rdf:type :Person 
```

Dieses Tripel ist implizit, es ist von den obigen fünf Tripel bedingt wird aber nicht explizit aufgeführt.

Führen Sie die folgende SPARQL Abfrage aus und beantworten Sie die Fragen.

In [14]:
reason(rdf, """
SELECT ?work ?title 
WHERE {
  ?work rdf:type :Work .
  ?work :title ?title .
}""")

# Erklären Sie warum Sie hier die drei Ressourcen erhalten obwohl Sie nach Werken suchen, 
# also nach der Oberklasse der beiden Klassen die Sie definiert haben.
#
# Anwort: Die drei Ressourcen sind Instanzen der Klassen :Album und :Single. Diese Klassen sind als Subklassen der Klasse :Work definiert. Somit sind die drei Ressourcen auch Instanzen der Klasse :Work, auch wenn "nur" implizit (also es kein explizites Tripel :r1 rdf:type :Work gibt). Das sogenannte reasoning erschliesst solche (logisch) implizite Tripel.

,work,title
0,http://example.org#r1,The Dark Side of the Moon
1,http://example.org#r2,The Wall
2,http://example.org#r3,"What God Wants, Part 1"
